In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 20 22:25:21 2020

@author: tuyen
"""
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 20 22:05:43 2020

@author: Lee
"""
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16

from keras.models import Model
import imutils
from imutils import paths
import numpy as np

import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt1
###############################################################################
from keras.preprocessing.image import img_to_array


###############################################################################
class FCHeadNet:
    @staticmethod
    def build(baseModel, classes, D):
        # initialize the head model that will be placed on top of
        # the base, then add a FC layer
        headModel = baseModel.output
        headModel = Flatten(name="flatten")(headModel)
        headModel = Dense(D, activation="relu")(headModel)
        headModel = Dropout(0.5)(headModel)

        # add a softmax layer
        headModel = Dense(classes, activation="softmax")(headModel)

        # return the model
        return headModel


###############################################################################
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode="nearest")

# grab the list of images that we’ll be describing, then extract
# the class label names from the image paths
print("[INFO] loading images...")

path_data = "D:/MCUT/Neural Network/datasets/animals"
imagePaths = list(paths.list_images(path_data))
# imagePaths = list(paths.list_images(args["dataset"]))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

# load our image dataset from disk
# initialize the image preprocessors
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixel intensities to
# the range [0, 1]
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=1)  ## verbose = 500
data = data.astype("float") / 255.0

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)
# performing network surgery

# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
num_epochs = 20
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
                        validation_data=(testX, testY),
                        epochs=num_epochs,
                        steps_per_epoch=len(trainX) // 32,
                        verbose=100)

model.save("E:/MCUT/Neural Networks/Day 4/before_fine_tuned_VGG_weights.hdf5")

plt.figure()
plt.plot(np.arange(0, num_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, num_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, num_epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, num_epochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("E:/MCUT/Neural Networks/Day 4/before_fine_tuningVGG16.png")
# evaluate the network after initialization
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=classNames))
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")

H1 = model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
                         validation_data=(testX, testY),
                         epochs=num_epochs,
                         steps_per_epoch=len(trainX) // 32,
                         verbose=1)
model.save("E:/MCUT/Neural Networks/Day 4/after_fine_tuned_VGG_weights.hdf5")
# evaluate the network on the fine-tuned model
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=classNames))

# # save the model to disk
# print("[INFO] serializing model...")
# save_model = "E:/Teaching/Books/Programs/datasets/hdf5_2/flood.model"
# model.save(save_model)

plt1.style.use("ggplot")

plt1.figure()
plt1.plot(np.arange(0, num_epochs), H1.history["loss"], label="train_loss")
plt1.plot(np.arange(0, num_epochs), H1.history["val_loss"], label="val_loss")
plt1.plot(np.arange(0, num_epochs), H1.history["accuracy"], label="train_acc")
plt1.plot(np.arange(0, num_epochs), H1.history["val_accuracy"], label="val_acc")
plt1.title("Training Loss and Accuracy")
plt1.xlabel("Epoch #")
plt1.ylabel("Loss/Accuracy")
plt1.legend()
plt1.savefig("E:/MCUT/Neural Networks/Day 4/after_fine_tuningVGG16.png")
plt1.show()
